## Building Q&A application using Knowledge Bases for Amazon Bedrock - Retrieve API

### Context

In this notebook, we will dive deep into building Q&A application using Knowledge Bases for Amazon Bedrock - Retrieve API. Here, we will query the knowledge base to get the desired number of document chunks based on similarity search. We will then augment the prompt with relevant documents and query which will go as input to Anthropic Claude V2 for generating response.

With a knowledge base, you can securely connect foundation models (FMs) in Amazon Bedrock to your company
data for Retrieval Augmented Generation (RAG). Access to additional data helps the model generate more relevant,
context-speciﬁc, and accurate responses without continuously retraining the FM. All information retrieved from
knowledge bases comes with source attribution to improve transparency and minimize hallucinations. For more information on creating a knowledge base using console, please refer to this [post](https://docs.aws.amazon.com/bedrock/latest/userguide/knowledge-base.html).
We will cover 2 parts in the notebook:
- Part 1, we will share how you can use `RetrieveAPI` with foundation models from Amazon Bedrock. We will use the `mistral.mistral-7b` model. 
- Part 2, we will showcase the langchain integration using Anthropic Claude 3 Haiku model.

### Pattern

We can implement the solution using Retreival Augmented Generation (RAG) pattern. RAG retrieves data from outside the language model (non-parametric) and augments the prompts by adding the relevant retrieved data in context. Here, we are performing RAG effectively on the knowledge base created using console/sdk. 

### Pre-requisite

Before being able to answer the questions, the documents must be processed and ingested in vector database.

1. Load the documents into the knowledge base by connecting your s3 bucket (data source). 
2. Ingestion - Knowledge bases will split them into smaller chunks (based on the strategy selected), generate embeddings and store it in the associated vectore store and notebook [0_create_ingest_documents_test_kb.ipynb](./0\_create_ingest_documents_test_kb.ipynb) takes care of it for you.

![data_ingestion](./images/data_ingestion.png)


#### Notebook Walkthrough



For our notebook we will use the `Retreive API` provided by Knowledge Bases for Amazon Bedrock which converts user queries into
embeddings, searches the knowledge base, and returns the relevant results, giving you more control to build custom
workﬂows on top of the semantic search results. The output of the `Retrieve API` includes the the `retrieved text chunks`, the `location type` and `URI` of the source data, as well as the relevance `scores` of the retrievals. 


We will then use the text chunks being generated and augment it with the original prompt and pass it through the `anthropic.claude-v2` model using prompt engineering patterns based on your use case.
    

### USE CASE:

#### Dataset

In this example, you will use several years of Amazon's Letter to Shareholders as a text corpus to perform Q&A on. This data is already ingested into the Knowledge Bases for Amazon Bedrock. You will need the `knowledge base id` to run this example.
In your specific use case, you can sync different files for different domain topics and query this notebook in the same manner to evaluate model responses using the retrieve API from knowledge bases.


### Python 3.10

⚠  For this lab we need to run the notebook based on a Python 3.10 runtime. ⚠

If you carry out the workshop from your local environment outside of the Amazon SageMaker studio please make sure you are running a Python runtime > 3.10.

### Setup

To run this notebook you would need to install following packages.


In [2]:
%pip install --force-reinstall -q -r ./requirements.txt

^C
ERROR: Operation cancelled by user
Note: you may need to restart the kernel to use updated packages.


In [3]:
%store -r

#### Restart the kernel with the updated packages that are installed through the dependencies above

In [4]:
# restart kernel
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

### Follow the steps below to initiate the bedrock client:

1. Import the necessary libraries, along with langchain for bedrock model selection, llama index to store the service context containing the llm and embedding model instances. We will use this service context later in the notebook for evaluating the responses from our Q&A application. 

2. Initialize `anthropic.claude-v2` as our large language model to perform query completions using the RAG pattern with the given knowledge base, once we get all text chunk searches through the `retrieve` API.

In [5]:
import boto3
import pprint
from botocore.client import Config
import json

pp = pprint.PrettyPrinter(indent=2)
session = boto3.session.Session()
region = session.region_name
bedrock_config = Config(connect_timeout=120, read_timeout=120, retries={'max_attempts': 0})
bedrock_client = boto3.client('bedrock-runtime', region_name = region)
bedrock_agent_client = boto3.client("bedrock-agent-runtime",
                              config=bedrock_config, region_name = region)
print(region)

us-east-1


### Part 1 - Retrieve API with foundation models from Amazon Bedrock

Define a retrieve function that calls the `Retreive API` provided by Knowledge Bases for Amazon Bedrock which converts user queries into
embeddings, searches the knowledge base, and returns the relevant results, giving you more control to build custom
workﬂows on top of the semantic search results. The output of the `Retrieve API` includes the the `retrieved text chunks`, the `location type` and `URI` of the source data, as well as the relevance `scores` of the retrievals. You can also use the  `overrideSearchType` option in `retrievalConfiguration` which offers the choice to use either `HYBRID` or `SEMANTIC`. By default, it will select the right strategy for you to give you most relevant results, and if you want to override the default option to use either hybrid or semantic search, you can set the value to `HYBRID/SEMANTIC`.

![retrieveAPI](./images/retrieveAPI.png)



In [6]:
def retrieve(query, kbId, numberOfResults=5):
    return bedrock_agent_client.retrieve(
        retrievalQuery= {
            'text': query
        },
        knowledgeBaseId=kbId,
        retrievalConfiguration= {
            'vectorSearchConfiguration': {
                'numberOfResults': numberOfResults,
                'overrideSearchType': "HYBRID", # optional
            }
        }
    )

#### Initialize your Knowledge base id before querying responses from the initialized LLM

Next, we will call the `retreive API`, and pass `knowledge base id`, `number of results` and `query` as paramters. 

`score`: You can view the associated score of each of the text chunk that was returned which depicts its correlation to the query in terms of how closely it matches it.

In [7]:
query = "What is Amazon doing in the field of generative AI?"
response = retrieve(query, kb_id, 5)
retrievalResults = response['retrievalResults']
pp.pprint(retrievalResults)

[ { 'content': { 'text': 'Imagine what they’ll be able to do with reliable '
                         'connectivity, from people taking online education '
                         'courses, using financial services, starting their '
                         'own businesses, doing their shopping, enjoying '
                         'entertainment, to businesses and governments '
                         'improving their coverage, efficiency, and '
                         'operations. Kuiper will deliver not only '
                         'accessibility, but affordability. Our teams have '
                         'developed low-cost antennas (i.e. customer '
                         'terminals) that will lower the barriers to access. '
                         'We recently unveiled the new terminals that will '
                         'communicate with the satellites passing overhead, '
                         'and we expect to be able to produce our standard '
                     

### Extract the text chunks from the retrieveAPI response

In the cell below, we will fetch the context from the retrieval results.

In [8]:
# fetch context from the response
def get_contexts(retrievalResults):
    contexts = []
    for retrievedResult in retrievalResults: 
        contexts.append(retrievedResult['content']['text'])
    return contexts

In [9]:
contexts = get_contexts(retrievalResults)
pp.pprint(contexts)

[ 'Imagine what they’ll be able to do with reliable connectivity, from people '
  'taking online education courses, using financial services, starting their '
  'own businesses, doing their shopping, enjoying entertainment, to businesses '
  'and governments improving their coverage, efficiency, and operations. '
  'Kuiper will deliver not only accessibility, but affordability. Our teams '
  'have developed low-cost antennas (i.e. customer terminals) that will lower '
  'the barriers to access. We recently unveiled the new terminals that will '
  'communicate with the satellites passing overhead, and we expect to be able '
  'to produce our standard residential version for less than $400 each. '
  'They’re small: 11 inches square, 1 inch thick, and weigh less than 5 pounds '
  'without their mounting bracket, but they deliver speeds up to 400 megabits '
  'per second. And they’re powered by Amazon-designed baseband chips. We’re '
  'preparing to launch two prototype satellites to test 

### Prompt specific to the model to personalize responses 

Here, we will use the specific prompt below for the model to act as a financial advisor AI system that will provide answers to questions by using fact based and statistical information when possible. We will provide the `Retrieve API` responses from above as a part of the `{contexts}` in the prompt for the model to refer to, along with the user `query`.  

In [10]:
prompt = f"""
Human: You are a financial advisor AI system, and provides answers to questions by using fact based and statistical information when possible. 
Use the following pieces of information to provide a concise answer to the question enclosed in <question> tags. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
<context>
{contexts}
</context>

<question>
{query}
</question>

The response should be specific and use statistics or numbers when possible.

Assistant:"""

### Invoke foundation model from Amazon Bedrock
In this example, we will use `mistral.mistral-7b` foundation model from Amazon Bedrock. 
Its a 7B dense Transformer, fast-deployed and easily customizable. Small, yet powerful for a variety of use cases.
- Max tokens: 8K
- Languages: English
- Supported use cases: Text summarization, structuration, question answering, and code completion

In [11]:
# payload with model paramters
mistral_payload = json.dumps({
    "prompt": prompt,
    "max_tokens":512,
    "temperature":0.5,
    "top_k":50,
    "top_p":0.9
})

<div class="alert alert-block alert-info">
<b>Note:</b> Please make sure you have subscribed to the <b>mistral-7B</b> model on the model access page in the Amazon Bedrock console before proceeding.
</div>

In [12]:
modelId = 'mistral.mistral-7b-instruct-v0:2' # change this to use a different version from the model provider
accept = 'application/json'
contentType = 'application/json'
response = bedrock_client.invoke_model(body=mistral_payload, modelId=modelId, accept=accept, contentType=contentType)
response_body = json.loads(response.get('body').read())
response_text = response_body.get('outputs')[0]['text']

pp.pprint(response_text)

(' Amazon is investing substantially in Generative AI, which is based on Large '
 'Language Models (LLMs) trained on hundreds of billions of parameters. They '
 'believe it will transform and improve virtually every customer experience. '
 'AWS is offering the most price-performant machine learning chips, Trainium '
 'and Inferentia, for small and large companies to affordably train and run '
 'their LLMs in production. AWS is also delivering applications like '
 'CodeWhisperer, which revolutionizes developer productivity by generating '
 'code suggestions in real time. Amazon plans to continue investing in these '
 'models across all of their consumer, seller, brand, and creator experiences.')


## Part 2 - LangChain integration
In this notebook, we will dive deep into building Q&A application using Retrieve API provided by Knowledge Bases for Amazon Bedrock and LangChain. We will query the knowledge base to get the desired number of document chunks based on similarity search, integrate it with LangChain retriever and use Anthropic Claude V2.1 model for answering questions.

In [13]:
from langchain_aws import ChatBedrock
from langchain.retrievers.bedrock import AmazonKnowledgeBasesRetriever

llm = ChatBedrock(model_id="anthropic.claude-3-haiku-20240307-v1:0",
              client = bedrock_client)

Create a `AmazonKnowledgeBasesRetriever` object from LangChain which will call the `Retreive API` provided by Knowledge Bases for Amazon Bedrock which converts user queries into embeddings, searches the knowledge base, and returns the relevant results, giving you more control to build custom workﬂows on top of the semantic search results. The output of the `Retrieve API` includes the the `retrieved text chunks`, the `location type` and `URI` of the source data, as well as the relevance `scores` of the retrievals.

In [14]:
query = "By what percentage did AWS revenue grow year-over-year in 2022?"
retriever = AmazonKnowledgeBasesRetriever(
        knowledge_base_id=kb_id,
        retrieval_config={"vectorSearchConfiguration": 
                          {"numberOfResults": 4,
                           'overrideSearchType': "SEMANTIC", # optional
                           }
                          },
        # endpoint_url=endpoint_url,
        # region_name=region,
        # credentials_profile_name="<profile_name>",
    )
docs = retriever.invoke(
        input=query
    )
pp.pprint(docs)

/tmp/ipykernel_3822/1691534166.py:2: LangChainDeprecationWarning: The class `AmazonKnowledgeBasesRetriever` was deprecated in LangChain 0.3.16 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-aws package and should be used instead. To use it run `pip install -U :class:`~langchain-aws` and import as `from :class:`~langchain_aws import AmazonKnowledgeBasesRetriever``.
  retriever = AmazonKnowledgeBasesRetriever(


[ Document(metadata={'location': {'s3Location': {'uri': 's3://bedrock-kb-us-east-1-183295442349/AMZN-2021-Shareholder-Letter.pdf'}, 'type': 'S3'}, 'score': 0.70448625, 'source_metadata': {'x-amz-bedrock-kb-source-uri': 's3://bedrock-kb-us-east-1-183295442349/AMZN-2021-Shareholder-Letter.pdf', 'x-amz-bedrock-kb-document-page-number': 1.0, 'x-amz-bedrock-kb-chunk-id': '1%3A0%3A97LQXZYB1hYXEzxD8RSg', 'x-amz-bedrock-kb-data-source-id': 'LHWUTR6IBF'}}, page_content='Many concluded that they didn’t want to continue managing their technology infrastructure themselves, and made the decision to accelerate their move to the cloud. This shift by so many companies (along with the economy recovering) helped re-accelerate AWS’s revenue growth to 37% YoY in 2021.     Conversely, our Consumer revenue grew dramatically in 2020. In 2020, Amazon’s North America and International Consumer revenue grew 39% YoY on the very large 2019 revenue base of $245 billion; and, this extraordinary growth extended into

## Prompt specific to the model to personalize responses
Here, we will use the specific prompt below for the model to act as a financial advisor AI system that will provide answers to questions by using fact based and statistical information when possible. We will provide the Retrieve API responses from above as a part of the `{context}` in the prompt for the model to refer to, along with the user `query`.

In [16]:
from langchain.prompts import PromptTemplate

PROMPT_TEMPLATE = """
Human: You are a financial advisor AI system, and provides answers to questions by using fact based and statistical information when possible. 
Use the following pieces of information to provide a concise answer to the question enclosed in <question> tags. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
<context>
{context}
</context>

<question>
{question}
</question>

The response should be specific and use statistics or numbers when possible.

Assistant: Hello, """
claude_prompt = PromptTemplate(template=PROMPT_TEMPLATE, 
                               input_variables=["context","question"])

### Integrating the retriever and the LLM defined above with RetrievalQA Chain to build the Q&A application.

In [17]:
from langchain.chains import RetrievalQA

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": claude_prompt}
)

In [18]:
answer = qa.invoke(query)
pp.pprint(answer)


{ 'query': 'By what percentage did AWS revenue grow year-over-year in 2022?',
  'result': 'Based on the context provided, the passage states that "despite '
            'growing 29% year-over-year ("YoY") in 2022 on a $62B revenue '
            'base, AWS faces short-term headwinds right now as companies are '
            'being more cautious in spending given the challenging, current '
            'macroeconomic conditions."\n'
            '\n'
            'So the answer is that AWS revenue grew 29% year-over-year in '
            '2022.',
  'source_documents': [ Document(metadata={'location': {'s3Location': {'uri': 's3://bedrock-kb-us-east-1-183295442349/AMZN-2021-Shareholder-Letter.pdf'}, 'type': 'S3'}, 'score': 0.70448625, 'source_metadata': {'x-amz-bedrock-kb-source-uri': 's3://bedrock-kb-us-east-1-183295442349/AMZN-2021-Shareholder-Letter.pdf', 'x-amz-bedrock-kb-document-page-number': 1.0, 'x-amz-bedrock-kb-chunk-id': '1%3A0%3A97LQXZYB1hYXEzxD8RSg', 'x-amz-bedrock-kb-data-source-

## Conclusion
You can use Retrieve API for customizing your RAG based application, using either `InvokeModel` API from Bedrock, or you can integrate with LangChain using `AmazonKnowledgeBaseRetriever`.
Retrieve API provides you with the flexibility of using any foundation model provided by Amazon Bedrock, and choosing the right search type, either HYBRID or SEMANTIC, based on your use case. 
Here is the [blog](#https://aws.amazon.com/blogs/machine-learning/knowledge-bases-for-amazon-bedrock-now-supports-hybrid-search/) for Hybrid Search feature, for more details.

<div class="alert alert-block alert-warning">
<b> Remember to CLEAN_UP at the end of your session. </b>
</div>